In [6]:
%pylab
import pandas as pd
import glob as glob

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


**Introduction**

The dataset from MS Birka Stockholm is in .xls Excel-97 format. And the data was gathered in several steps during three different trips. Some of the data is overlapping in time-index, and same headers (data points) exist in several files. So to be able to filter and consolidate all the data it must be done in several steps. As the Excel-97 format is limited in 65k rows and also a limited amount of columns it was needed to divide into several files.

Some of the data is in Boolean format, and some have data-points missing but the majority should be in numerical format.

In all Excel-files the meta data of each data-point (header) is in the first 14 rows. The first step is to make a pre-processing of the .xls files, and filter out non uni-code characters, put in a split character between the meta-data and joining everything in the data header. Still keeping the index in time-series format.


In [7]:
csv_data_path = '/Users/fredde/Database/csv-1year/'
xls_data_path = '/Users/fredde/Database/data-files/1year/'
database_path = '/Users/fredde/Database/'

xlsfiles = glob.glob(xls_data_path + '*.xls')
print(xlsfiles)
df = pd.DataFrame()
all_data = pd.DataFrame()

['/Users/fredde/Database/data-files/1year/1 year 15m int.xls', '/Users/fredde/Database/data-files/1year/1-1161-2203.xls', '/Users/fredde/Database/data-files/1year/12mo 15m int fuel rack ship speed ae me etc.xls', '/Users/fredde/Database/data-files/1year/12mo 15min interval fuel oil tank.xls', '/Users/fredde/Database/data-files/1year/1year_mean_motor_fuel rack_etc.xls', '/Users/fredde/Database/data-files/1year/2-2252-3347.xls', '/Users/fredde/Database/data-files/1year/3-3364-4641.xls', '/Users/fredde/Database/data-files/1year/5-6437-40006.xls', '/Users/fredde/Database/data-files/1year/prio2.xls', '/Users/fredde/Database/data-files/1year/prio3.xls', '/Users/fredde/Database/data-files/1year/prio4-1101-2112.xls', '/Users/fredde/Database/data-files/1year/prio4-2201-3652.xls', '/Users/fredde/Database/data-files/1year/prio4-3701-end.xls', '/Users/fredde/Database/data-files/1year/scr me ae.xls', '/Users/fredde/Database/data-files/1year/ship speed outside temp.xls']


Clean up csv

In [3]:
# Make CSV-files and clean up headers.

for i in range(len(xlsfiles)):
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    
    print('Processing: '+str(xlsfiles[i].split('/')[-1].split('.')[0]))
    df = pd.read_excel(xlsfiles[i],index_col=0)
    
    headers = list(df)
    headers_new = list()
    
    # And now extract the relevant meta-data in the first couple of rows.
    # Make a new list of headers in the file. Using ':' as a split.
    

    for head in headers:
    
        name = str(df[head].iloc[0])
        id_nr = str(head.split('.')[2].split(':')[1])
        unit = str(df[head].iloc[1])
        data_type = str(df[head].iloc[5])
        sample_interval = str(df[head].iloc[8])

        headers_new.append(str(name+':'+id_nr+':'+unit+':'+data_type+':'+sample_interval))
        
    for n in range(len(headers_new)):
        series = df[headers[n]].ix[13:]
        df2[headers_new[n]] = series
    
    # Save in .csv format.
    df2.to_csv(csv_data_path + xlsfiles[i].split('/')[-1].split('.')[0] + '.csv')
    #df2.to_excel(csv_data_path + xlsfiles[i].split('/')[-1].split('.')[0] + '.xls')
    
    # Clean up memory
    del df2
    del df
    print(str(i+1) + ' done of ' + str(len(xlsfiles)))
    
print('All done!')

Processing: 1 year 15m int
1 done of 15
Processing: 1-1161-2203
2 done of 15
Processing: 12mo 15m int fuel rack ship speed ae me etc
3 done of 15
Processing: 12mo 15min interval fuel oil tank
4 done of 15
Processing: 1year_mean_motor_fuel rack_etc
5 done of 15
Processing: 2-2252-3347
6 done of 15
Processing: 3-3364-4641
7 done of 15
Processing: 5-6437-40006
8 done of 15
Processing: prio2
9 done of 15
Processing: prio3
10 done of 15
Processing: prio4-1101-2112
11 done of 15
Processing: prio4-2201-3652
12 done of 15
Processing: prio4-3701-end
13 done of 15
Processing: scr me ae
14 done of 15
Processing: ship speed outside temp
15 done of 15
All done!


Time to create a database, joining all the files into one master DataFrame.

In [3]:
all_data=pd.DataFrame()
csvfiles = glob.glob(csv_data_path + '*.csv')


for i in range(len(csvfiles)):
    df = pd.DataFrame()
    print('Processing: '+str(csvfiles[i].split('/')[-1].split('.')[0]))
    df = pd.read_csv(csvfiles[i],header=0,index_col=0,dtype='a')
    all_data = all_data.append(df)
    
    del df # Clean up memory
    print(str(i+1) + ' done of ' + str(len(csvfiles)))

df = all_data.sort_index()

del all_data # Clean memory
df_out = pd.DataFrame() # Make a new DataFrame so the process of converting to numeric goes faster

for i in range(len(list(df))):
    df_out[list(df)[i]] = pd.to_numeric(df[list(df)[i]],errors='ignore')

df_out.to_hdf(database_path + 'all_data_1year.h5','table')

print('All done!')

Processing: 1 year 15m int
1 done of 15
Processing: 1-1161-2203
2 done of 15
Processing: 12mo 15m int fuel rack ship speed ae me etc
3 done of 15
Processing: 12mo 15min interval fuel oil tank
4 done of 15
Processing: 1year_mean_motor_fuel rack_etc
5 done of 15
Processing: 2-2252-3347
6 done of 15
Processing: 3-3364-4641
7 done of 15
Processing: 5-6437-40006
8 done of 15
Processing: prio2
9 done of 15
Processing: prio3
10 done of 15
Processing: prio4-1101-2112
11 done of 15
Processing: prio4-2201-3652
12 done of 15
Processing: prio4-3701-end
13 done of 15
Processing: scr me ae
14 done of 15
Processing: ship speed outside temp
15 done of 15


/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['AE1 RUNNING:15004:-:Raw data:900', 'AE1 RUNNING:1769:-:Raw data:900', 'AE2 RUNNING:25004:-:Raw data:900', 'AE2 RUNNING:2769:-:Raw data:900', 'AE3 RUNNING:35004:-:Raw data:900', 'AE3 RUNNING:3769:-:Raw data:900', 'AE4 RUNNING:45004:-:Raw data:900', 'AE4 RUNNING:4769:-:Raw data:900', 'FO VISCOSITY ME1/3:6141:-:Raw data:900', 'FO VISCOSITY ME2/4:6142:-:Raw data:900', 'ME1 RUNNING:10004:-:Raw data:900', 'ME1 TC BY-PASS VALVE:1368:-:Raw data:900', 'ME1/3 BOOST FO VLV:61111:-:Raw data:900', 'ME1/3|AE1/3 BOOST UN:6139:-:Raw data:900', 'ME2 RUNNING:20004:-:Raw data:900', 'ME2 TC BY-PASS VALVE:2368:-:Raw data:900', 'ME2/4 BOOST FO VLV:61121:-:Raw data:900', 'ME2/4|AE2/4 BOOST UN:6140:-:Raw data:900', 'ME3 RUNNING:30004:-:Raw data:900', 'ME3 TC BY-PASS VAL

All done!


Load the data and see if it can be used..

In [12]:
df = pd.read_hdf(database_path + 'all_data_1year.h5','table')



NameError: name 'blosc' is not defined

In [13]:
df.to_hdf(database_path + 'all_data_1year_comp.h5','table',complevel=9,complib='blosc')

/anaconda/lib/python3.6/site-packages/pandas/core/generic.py:1138: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['AE1 RUNNING:15004:-:Raw data:900', 'AE1 RUNNING:1769:-:Raw data:900', 'AE2 RUNNING:25004:-:Raw data:900', 'AE2 RUNNING:2769:-:Raw data:900', 'AE3 RUNNING:35004:-:Raw data:900', 'AE3 RUNNING:3769:-:Raw data:900', 'AE4 RUNNING:45004:-:Raw data:900', 'AE4 RUNNING:4769:-:Raw data:900', 'FO VISCOSITY ME1/3:6141:-:Raw data:900', 'FO VISCOSITY ME2/4:6142:-:Raw data:900', 'ME1 RUNNING:10004:-:Raw data:900', 'ME1 TC BY-PASS VALVE:1368:-:Raw data:900', 'ME1/3 BOOST FO VLV:61111:-:Raw data:900', 'ME1/3|AE1/3 BOOST UN:6139:-:Raw data:900', 'ME2 RUNNING:20004:-:Raw data:900', 'ME2 TC BY-PASS VALVE:2368:-:Raw data:900', 'ME2/4 BOOST FO VLV:61121:-:Raw data:900', 'ME2/4|AE2/4 BOOST UN:6140:-:Raw data:900', 'ME3 RUNNING:30004:-:Raw data:900', 'ME3 TC BY-PASS VAL

In [14]:
df2 = pd.read_hdf(database_path + 'all_data_1year_comp.h5','table')

In [16]:
df2.describe()

,AE SCR 2CFC20 ALARM:6854:-:Average:900,AE SCR CBD10 230V IS:6858:-:Average:900,AE SCR CBD10 24V ISO:6857:-:Average:900,AE SCR CBD10 FUSE:6856:-:Average:900,AE SCR CBD10 WARNING:6853:-:Average:900,AE SCR CFC20 230V IS:6868:-:Average:900,AE SCR CFC20 AIR CON:6874:-:Average:900,AE SCR CFC20 ANALYS:6867:-:Average:900,AE SCR CFC20 HUMIDIT:6873:-:Average:900,AE SCR CFC20 MEAS GA:6871:-:Average:900,...,SW PUMP1 ME/AE1/3 PR:6402:bar:Average:900,SW PUMP2 AUX EQU PRE:6407:bar:Average:900,SW PUMP2 ME/AE1/3 PR:6403:bar:Average:900,SW PUMP3 ME/AE2/4 PR:6404:bar:Average:900,SW PUMP4 ME/AE2/4 PR:6405:bar:Average:900,SW TEMP TO ME/AE 1/3:6410:C:Average:900,SW TEMP TO ME/AE 2/4:6411:C:Average:900,TECHNICAL WAT T T8CS:6705:m3:Average:900,UREA TANK T42P:6802:m3:Average:900,UREA TANK T42S:6801:m3:Average:900
count,36652.000000,36652.0,36652.0,36652.0,36652.0,36652.0,36652.000000,36652.0,36652.000000,36652.0,...,36652.000000,36652.000000,36652.000000,36652.000000,36652.000000,36652.000000,36652.000000,36652.000000,36652.000000,36652.000000
mean,-0.004376,0.0,0.0,0.0,0.0,0.0,-0.002803,0.0,-0.000198,0.0,...,0.831218,0.978587,0.828737,0.837403,0.849842,17.406551,17.626834,28.329551,28.523836,31.065560
std,0.065139,0.0,0.0,0.0,0.0,0.0,0.052230,0.0,0.012491,0.0,...,0.068473,0.383757,0.068858,0.063015,0.063320,2.480810,2.109453,5.728018,11.957578,11.717892
min,-1.000000,0.0,0.0,0.0,0.0,0.0,-1.000000,0.0,-1.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.812698,0.708311,0.810094,0.811461,0.824225,16.021978,16.498169,25.966667,18.680001,23.078333
50%,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.830346,0.818201,0.827937,0.841807,0.853659,17.076925,17.252748,29.520000,31.126667,33.813334
75%,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.851705,1.090256,0.849687,0.865055,0.878340,18.241758,18.461538,32.226668,38.633334,40.200001
max,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,3.016622,2.141734,3.015320,2.765193,2.779520,28.461538,27.391942,39.166668,54.400002,54.400002


In [20]:
%timeit df['AE SCR 2CFC20 ALARM:6854:-:Average:900']

The slowest run took 13.03 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.17 µs per loop


In [19]:
%timeit df2['AE SCR 2CFC20 ALARM:6854:-:Average:900']

The slowest run took 11.69 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.13 µs per loop
